In [2]:
from phase2 import *
import os
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

In [9]:
sample_size = 3 # clusters must have at least 3 parcels
min_urban_distance = 15 # min. distance between two likely neighbors
max_distance = 200 # max. distance between two likely neighbors

In [ ]:

data_dir = r'D:\Projects\superparcels\data\phase2'
print('Reading data...')
place_gdf = gpd.read_file(os.path.join(data_dir, 'alameda_test_place.shp'))[['PLACEFP', 'geometry']]
utm_crs = place_gdf.estimate_utm_crs().to_epsg()
place_gdf = place_gdf.to_crs(epsg=utm_crs)
place_gdf = place_gdf.loc[place_gdf['PLACEFP'] == '41992']
parcels = gpd.read_file(os.path.join(data_dir, 'sp_sample_06001_cluster_canidates.shp'))
parcels = parcels.to_crs(epsg=utm_crs)


Reading data...


In [10]:
all_clustered_parcel_data = gpd.GeoDataFrame()
all_single_parcel_data = gpd.GeoDataFrame()
for place_id, place_data in place_gdf.iterrows():
    place_id += 1
    print(f"Processing place {place_id}")
    sub_parcels = parcels[parcels.intersects(place_data['geometry'])]
    print('Computing distance matrix...')
    distances = compute_regional_distance_matrix(sub_parcels)
    print('Computing density...')
    density = compute_density(distances, len(sub_parcels))
    print('Computing n_neighbors...')
    min_nneighbors = max(0.1 * len(sub_parcels), 3) # 10% of the total number of parcels
    n_neighbors = compute_nneighbors(density, min_nneighbors)

Processing place 2
Computing distance matrix...
Computing density...
Computing n_neighbors...


In [14]:
len(sub_parcels)

4753

In [15]:
n_neighbors

1303

In [16]:
min_nneighbors


475.3

In [17]:
distances

array([2533.20887899, 1994.74615058,  978.8558395 , ..., 2804.146132  ,
       2048.8089824 , 5632.79064995])

In [19]:
len(distances)

11291145

In [21]:
knn = NearestNeighbors(n_neighbors=n_neighbors).fit(distances.reshape(-1, 1))
knn_distances, _ = knn.kneighbors(distances.reshape(-1, 1))

MemoryError: Unable to allocate 110. GiB for an array with shape (11291145, 1303) and data type float64

In [ ]:
knn = NearestNeighbors(n_neighbors=n_neighbors).fit(dmatrix.reshape(-1, 1))
knn_distances, _ = knn.kneighbors(dmatrix.reshape(-1, 1))
sorted_distances = np.sort(knn_distances[:, -1])
smooth_dist = uniform_filter1d(sorted_distances, size=10)
difference = np.diff(smooth_dist)
elbow_index = np.argmax(difference) + 1

In [ ]:
optimal_distance, knn_optimal_distance = compute_optimal_distance(distances, n_neighbors, min_urban_distance, max_distance)
